# 6. Cultuurconnect - Extracting information from the xml's

In [3]:
import pandas as pd
import xml_info_cultuurconnect as cultuurconnect
import lxml.etree as ET
import json

# Availability - holding information

### 1. Loading availability information

In [4]:
with open('cleaning/data/cultuurconnect_availability_total.json','r') as f:
        productie_dict = json.load(f)

In [5]:
df = pd.DataFrame.from_dict(productie_dict, orient='index', columns = {'xml_response'}).reset_index()
df.rename(columns = {'index':'frabl'}, inplace = True)

In [6]:
df.shape

(9271, 2)

In [7]:
df

,frabl,xml_response
0,17903665FF1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
1,7910F9AA58F1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
2,5561D93C4EF1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
3,39BC36FCFDF1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
4,40380C150F11AC31,"<aquabrowser version=""1"" before-rendering-time..."
...,...,...
9266,5A70034BD091AC31,"<aquabrowser version=""1"" before-rendering-time..."
9267,31181BB58FF1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
9268,422CB171F5F1ACA0,"<aquabrowser version=""1"" before-rendering-time..."
9269,76FB042B33F1AC31,"<aquabrowser version=""1"" before-rendering-time..."


### 2. Getting library count and holding count

In [8]:
df['holding_count'] = df['xml_response'].apply(lambda x: cultuurconnect.get_improved_holding_count(x))
df['library_list'] = df.xml_response.apply(lambda x: cultuurconnect.get_improved_library_list(x))
df['library_count'] = df.library_list.apply(lambda x: cultuurconnect.get_library_count(x))
df['library_list']=df.library_list.apply(lambda x: ";".join(x) if isinstance(x,list) else x)
df['improved_holding_hasselt']= df["xml_response"].apply(lambda x: cultuurconnect.get_improved_holding_count_Hasselt(x))
df['improved_library_list_hasselt'] = df.xml_response.apply(lambda x: cultuurconnect.get_improved_library_list_Hasselt(x))
df['improved_library_count_hasselt'] = df.improved_library_list_hasselt.apply(lambda x: cultuurconnect.get_library_count(x))
df['improved_library_list_hasselt']=df.improved_library_list_hasselt.apply(lambda x: ";".join(x) if isinstance(x,list) else x)

In [9]:
df.tail(2)

,frabl,xml_response,holding_count,library_list,library_count,improved_holding_hasselt,improved_library_list_hasselt,improved_library_count_hasselt
9269,76FB042B33F1AC31,"<aquabrowser version=""1"" before-rendering-time...",4.0,Hoofdbib Brasschaat;Hasselt Dusart;Bibliotheek...,3,3.0,Hoofdbib Brasschaat;Bibliotheek De Krook,2
9270,331993A5EAF1ACA7,"<aquabrowser version=""1"" before-rendering-time...",7.0,Hasselt Dusart;Provinciaal centrum voor cultur...,6,6.0,Provinciaal centrum voor cultureel erfgoed;Cen...,5


### Save intermediate dataframe

In [10]:
df.to_csv("cleaning/data/cultuurconnect_holding_information.csv")

In [11]:
holdings = pd.read_csv("cleaning/data/cultuurconnect_holding_information.csv", index_col=0, dtype={'isbn':str})

In [12]:
holdings.shape

(9271, 8)

# Details - metadata

### 1. Loading details information

In [15]:
with open("cleaning/data/cultuurconnect_details_total.json", 'r') as f:
    details_dict = json.load(f)

In [21]:
df = pd.DataFrame.from_dict(details_dict, orient='index', columns = {'xml_response'}).reset_index()
df.rename(columns = {'index':'frabl'}, inplace = True)

### 2. Getting the metadata from the xml's

In [22]:
df['tree'] = df.xml_response.apply(lambda x: ET.ElementTree(ET.fromstring(x)))

In [23]:
df['title'] = df.tree.apply(lambda x: cultuurconnect.get_title(x))
df['year'] = df.tree.apply(lambda x: cultuurconnect.get_publicationyear(x))
df['main_author'] = df.tree.apply(lambda x: cultuurconnect.get_main_author(x))
df['type'] = df.tree.apply(lambda x: cultuurconnect.get_type(x))
df['language'] = df.tree.apply(lambda x: cultuurconnect.get_language(x))
df['publisher'] = df.tree.apply(lambda x: cultuurconnect.get_publisher(x))
df['subject'] = df.tree.apply(lambda x: cultuurconnect.get_subject(x))

In [24]:
df.drop(columns = "tree", inplace = True)
df.to_csv("cleaning/data/cultuurconnect_details_information.csv")